In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
from pathlib import Path

import torch

from transformers import AutoModelForCausalLM
from peft import PeftModel

import sys
sys.path.insert(0, str("/fsx/home-augustas/mlmi-thesis/src/ppo"))
from utils import get_tokenizer

In [4]:
device = torch.device("cuda:3")
device

device(type='cuda', index=3)

In [11]:
tokenizer = get_tokenizer("lmsys/vicuna-7b-v1.5")
tokenizer

Loading tokenizer lmsys/vicuna-7b-v1.5...
Loaded tokenizer.



LlamaTokenizerFast(name_or_path='lmsys/vicuna-7b-v1.5', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "lmsys/vicuna-7b-v1.5",
    low_cpu_mem_usage=True,
    device_map={ "": device },
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# model

In [8]:
# peft_weight_path = Path("/fsx/home-augustas/ppo_logs/vicuna_UQA_3b_qnli_20230803_144559_52437/checkpoints/model_step_1_32")
peft_weight_path = Path("/fsx/home-augustas/ppo_logs/vicuna-v1.5_UQA_3b_qnli_20230805_144210_53882/checkpoints/model_step_1_64")

model = PeftModel.from_pretrained(model, peft_weight_path)
model = model.merge_and_unload()

In [9]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [10]:
# Push to hub
output_name = "vicuna-v1.5-rl-qnli-v1"

# model.save_pretrained(output_name)
# tokenizer.save_pretrained(output_name)

model.push_to_hub(output_name, private=True)
tokenizer.push_to_hub(output_name, private=True)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AugustasM/vicuna-v1.5-rl-qnli-v1/commit/9817ef73aeeb238edfcdc2d9be948f9c405bc7d5', commit_message='Upload tokenizer', commit_description='', oid='9817ef73aeeb238edfcdc2d9be948f9c405bc7d5', pr_url=None, pr_revision=None, pr_num=None)